In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')
import os 
import time
import numpy as np
from utils.gsheets_importer import path_finder

In [3]:
# Path to suite2p (if git cloned to code folder rather than pip installed)
s2p_path = os.path.expanduser('~/Documents/code/suite2p')
sys.path.append(s2p_path)
from suite2p.run_s2p import run_s2p

In [4]:
# default settings for analysis

ops = {
        'batch_size': 200, # reduce if running out of RAM
        'fast_disk': os.path.expanduser('~/Documents/suite2p_binaries'), # used to store temporary binary file, defaults to save_path0 (set as a string NOT a list)
         #'save_path0': '/media/jamesrowland/DATA/plab/suite_2p', # stores results, defaults to first item in data_path
        'delete_bin': False, # whether to delete binary file after processing
        # main settings
        'nplanes' : 1, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'diameter': 12, # this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
        'tau':  1.26, # this is the main parameter for deconvolution (1.25-1.5 for gcamp6s)
        'fs': 30.,  # sampling rate (total across planes)
        # output settings
        'save_mat': True, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        # parallel settings
        'num_workers': 0, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
        'num_workers_roi': 0, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
        # registration settings
        'do_registration': True, # whether to register data
        'nimg_init': 200, # subsampled frames for finding reference image
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': False, # whether to save registered tiffs
        'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
        # cell detection settings
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'navg_frames_svd': 5000, # max number of binned frames for the SVD
        'nsvd_for_roi': 1000, # max number of SVD components to keep for ROI detection
        'max_iterations': 20, # maximum number of iterations to do cell detection
        'ratio_neuropil': 6., # ratio between neuropil basis size and cell radius
        'ratio_neuropil_to_cell': 3, # minimum ratio between neuropil radius and cell radius
        'tile_factor': 1., # use finer (>1) or coarser (<1) tiles for neuropil estimation during cell detection
        'threshold_scaling': 1., # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'outer_neuropil_radius': np.inf, # maximum neuropil radius
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        # deconvolution settings
        'baseline': 'maximin', # baselining mode
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
      }

# make the local suite2p binaries file if it does not already exist
if not os.path.exists(ops['fast_disk']):
    os.mkdir(ops['fast_disk'])

In [12]:
# experiment specific settings
# can overwrite any parameter in default ops -- some examples shown below

# umbrella folder of the tiffs
data_path = '/mnt/qnap_share/Data/suite2p/'
# list of the tiffs you want to stitch together
# (Have hardcoded paths as this is clearer, could also join tiff names to data_path)
tiff_list = [  
    '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_000.tif',
    '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_001.tif',
    '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_002.tif',
    '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_003.tif',
    '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_004.tif',
            ]

# name of the folder to save results in (default = suite2p in data_path)
save_folder = os.path.expanduser('~/suite2p_results')
if not os.path.exists(save_folder):
    os.mkdir(save_folder)
    
diameter = 11  # the average diameter (in pixels) of a cell -- check in fiji 
fs = 30  # sampling rate of imaging (default 30 fps)
nplanes = 1  # number of planes (default 1)
nchannels = 1 # number of channels aquired (default 1)  

db = {
     'data_path': data_path,
     'tiff_list': tiff_list, 
     'diameter': diameter, 
     'fs': fs,
     'nplanes': nplanes,
     'nchannels': nchannels,
     'save_folder': save_folder
     }

In [13]:
t1 = time.time()
opsEnd=run_s2p(ops=ops,db=db)
t2 = time.time()
print('Total time this cell was running is {}'.format(t2-t1))

{'data_path': '/mnt/qnap_share/Data/suite2p/', 'tiff_list': ['/mnt/qnap_share/Data/suite2p/M0_2017-10-13_000.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_001.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_002.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_003.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_004.tif'], 'diameter': 11, 'fs': 30, 'nplanes': 1, 'nchannels': 1, 'save_folder': '/home/jrowland/suite2p_results'}
['/mnt/qnap_share/Data/suite2p/M0_2017-10-13_000.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_001.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_002.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_003.tif', '/mnt/qnap_share/Data/suite2p/M0_2017-10-13_004.tif']
** Found 5 tifs - converting to binary **
800 frames of binary, time 4.97 sec.
1600 frames of binary, time 8.37 sec.
2400 frames of binary, time 11.90 sec.
3200 frames of binary, time 15.40 sec.
4000 frames of binary, time 18.95 sec.
4800 frames of binary, time 25.36 sec.
5600 frames of bina

KeyboardInterrupt: 